In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import os
print(f'pandas version: {pd.__version__}')

pd.set_option('max_columns', 500)
# pd.set_option('max_rows', 10)

pandas version: 1.1.3


In [2]:
"""
Load content of a DBF file into a Pandas data frame.

The iter() is required because Pandas doesn't detect that the DBF
object is iterable.
"""

from dbfread import DBF

## Check record timeocb > 1

In [3]:
set_raw_data_path = 'D:\\Users\\2193033\\Desktop\\cal_new_indices\\raw_data_2020' + '\\'
print(f'raw data path : {set_raw_data_path}')

raw data path : D:\Users\2193033\Desktop\cal_new_indices\raw_data_2020\


In [4]:
raw_data_file_names = os.listdir(set_raw_data_path)
raw_data_file_names

['1.data.dbf',
 '10.data.dbf',
 '11.data.dbf',
 '2.data.dbf',
 '3.data.dbf',
 '4.data.dbf',
 '5.data.dbf',
 '6.data.dbf',
 '7.data.dbf',
 '8.data.dbf',
 '9.data.dbf',
 'data_260963.dbf',
 'temp.DBF']

In [5]:
regex = r'^AZ.*'

for file_name in raw_data_file_names[4:5]:
    dbf = DBF(set_raw_data_path + file_name)
    df = pd.DataFrame(iter(dbf))
    
    df_data = pd.DataFrame(df['DATE'].unique(), columns=['DATE'])
    df_temp = df[(df['TIMEOCB']>1) & ~(df['FEEDER'].astype(str).isin([''])) & ~(df['FEEDER'].astype(str).str.contains(regex))]['DATE']\
        .value_counts().sort_index().reset_index().rename(columns={'index': 'DATE', 'DATE': 'Sustain_count'})
    df_data = df_data.merge(df_temp, how='left', on='DATE').fillna(0)
    df_data['Sustain_count'] = df_data['Sustain_count'].astype(int)
    print(df_data)
    
    input()


          DATE  Sustain_count
0   2020-03-01              0
1   2020-03-02              1
2   2020-03-03              0
3   2020-03-04              3
4   2020-03-05              4
5   2020-03-06              4
6   2020-03-07             34
7   2020-03-08              4
8   2020-03-09              2
9   2020-03-10              3
10  2020-03-11              0
11  2020-03-12              2
12  2020-03-13              1
13  2020-03-14              1
14  2020-03-15             35
15  2020-03-16              2
16  2020-03-17              1
17  2020-03-18              1
18  2020-03-19              0
19  2020-03-20              1
20  2020-03-21              0
21  2020-03-22              0
22  2020-03-23              2
23  2020-03-24              2
24  2020-03-25              0
25  2020-03-26              1
26  2020-03-27              1
27  2020-03-28              0
28  2020-03-29              0
29  2020-03-30              1
30  2020-03-31              1



In [6]:
df.shape

(1375, 42)

In [ ]:
df['DATE'] = pd.to_datetime(df['DATE'], errors='coerce')
df[(df['DATE'].dt.day==10) & (df['TIMEOCB']>1) & ~(df['FEEDER'].astype(str).isin(['']))]

In [ ]:
regex = r'^AZ.*'
df[df['FEEDER'].astype(str).str.contains(regex)]

## Append file

In [21]:
set_path = 'D:\\Users\\2193033\\Desktop\\cal_new_indices\\032020'
print(f'Path : {set_path}')

Path : D:\Users\2193033\Desktop\cal_new_indices\032020


In [25]:
folder_names = os.listdir(set_path)
folder_names

['1 2',
 '10',
 '11',
 '12 13',
 '15',
 '16 MAR 2020',
 '18 MAR 2020',
 '19-20 MAR 2020',
 '21 22 23',
 '25 26',
 '28 29 30',
 '3',
 '31',
 '4',
 '5',
 '6 7 14 17 24 27',
 '8',
 '9']

In [23]:
folder_names[-1:]

['9']

In [26]:
path = set_path + '/'

for name in folder_names[:]:
    print(path + name + '/data.dbf')
    print(name)
    dbf = DBF(path + name + '/data.dbf')
    df_data = pd.DataFrame(iter(dbf))
    print('\ndata')
    print(df_data['DATE'].value_counts().sort_index())
        
    dbf = DBF(path + name + '/switching.dbf')
    df_sw = pd.DataFrame(iter(dbf))
    print('\nswitching')
    if df_sw.shape[0]>0:
#         df_sw.drop(columns=['_NullFlags'], inplace=True)
        df_sw['tmp_date'] = df_sw['DATA_PK'].map(lambda x: str(x)[0:8])
        df_sw['month'] = df_sw['DATA_PK'].map(lambda x: int(str(x)[4:6]))
        df_sw['day'] = df_sw['DATA_PK'].map(lambda x: int(str(x)[6:8]))
        print(df_sw['tmp_date'].value_counts().sort_index())
        
    dbf = DBF(path + name + '/customer.dbf')
    df_cus = pd.DataFrame(iter(dbf))
    print('\ncustomer')
    if df_cus.shape[0]>0:
        df_cus['tmp_date'] = df_cus['ID_CUST'].map(lambda x: str(x)[1:9])
        df_cus['month'] = df_cus['ID_CUST'].map(lambda x: int(str(x)[5:7]))
        df_cus['day'] = df_cus['ID_CUST'].map(lambda x: int(str(x)[7:9]))
        print(df_cus['tmp_date'].value_counts().sort_index())
    
    temp = input()

D:\Users\2193033\Desktop\cal_new_indices\032020/1 2/data.dbf
1 2

data
2020-03-01    49
2020-03-02    54
Name: DATE, dtype: int64

switching
20200302    3
Name: tmp_date, dtype: int64

customer
20200302    3
Name: tmp_date, dtype: int64

D:\Users\2193033\Desktop\cal_new_indices\032020/10/data.dbf
10

data
2020-03-10    27
Name: DATE, dtype: int64

switching
20200310    12
Name: tmp_date, dtype: int64

customer
20200310    12
Name: tmp_date, dtype: int64

D:\Users\2193033\Desktop\cal_new_indices\032020/11/data.dbf
11

data
2020-03-11    26
Name: DATE, dtype: int64

switching

customer

D:\Users\2193033\Desktop\cal_new_indices\032020/12 13/data.dbf
12 13

data
2020-03-12    20
2020-03-13    22
Name: DATE, dtype: int64

switching
20200312    5
20200313    2
Name: tmp_date, dtype: int64

customer
20200312    3
20200313    1
Name: tmp_date, dtype: int64

D:\Users\2193033\Desktop\cal_new_indices\032020/15/data.dbf
15

data
2020-03-15    320
Name: DATE, dtype: int64

switching
20200315    99


In [ ]:
df_data['DATE'] = pd.to_datetime(df_data['DATE'], errors='coerce')
# df_data[(df_data['TIMEOCB']>1) & ~(df_data['FEEDER'].astype(str).isin([''])) & (df_data['DATE'].dt.day==18)]
df_data[(df_data['TIMEOCB']>1) & ~(df_data['FEEDER'].astype(str).isin(['']))]

In [ ]:
df_sw['DATE_FROM'] = pd.to_datetime(df_sw['DATE_FROM'], errors='coerce')
df_sw[df_sw['day']==18]

In [ ]:
df_cus[df_cus['day']==18]

In [ ]:
df_sw[df_sw['day']==11].merge(df_cus[df_cus['day']==11], how='left', on='ID_SW').sort_index()

In [ ]:
df_data[(df_data['TIMEOCB']>1) & ~(df_data['FEEDER'].astype(str).isin(['']))]['DATE'].value_counts().sort_index()

In [ ]:
df_sw.info()
df_sw['DATE_FROM'].value_counts().sort_index()

In [ ]:
df_data.to_csv('data.csv', index=False, header=False)

In [ ]:
!type data.csv

## FIle gathering

### data.dbf

In [7]:
set_path = 'D:\\Users\\2193033\\Desktop\\cal_new_indices\\032020'
print(f'Path : {set_path}')

Path : D:\Users\2193033\Desktop\cal_new_indices\032020


In [8]:
folder_names = os.listdir(set_path)
folder_names

['1 2',
 '10',
 '11',
 '12 13',
 '15',
 '16 MAR 2020',
 '18 MAR 2020',
 '19-20 MAR 2020',
 '21 22 23',
 '25 26',
 '28 29 30',
 '3',
 '31',
 '4',
 '5',
 '6 7 14 17 24 27',
 '8',
 '9',
 'offcial']

In [9]:
# folder_iters = folder_names[:1], folder_names[10:11], folder_names[19:20], folder_names[21:27], folder_names[1:10], folder_names[11:19], folder_names[20:21] # for 062020
# folder_iters = folder_names[:1], folder_names[12:13], folder_names[1:2], folder_names[-4:-3], folder_names[2:5], folder_names[-3:-1], folder_names[5:12], folder_names[13:24] # for 072020
# folder_iters = folder_names[4:5], folder_names[12:13], folder_names[-6:-5], folder_names[-4:-2], folder_names[0:4], folder_names[5:12], folder_names[13:22], folder_names[-5:-4] # for 092020
folder_iters = folder_names[:5], folder_names[5:-1] # for 032020
folder_iters

(['1 2', '10', '11', '12 13', '15'],
 ['16 MAR 2020',
  '18 MAR 2020',
  '19-20 MAR 2020',
  '21 22 23',
  '25 26',
  '28 29 30',
  '3',
  '31',
  '4',
  '5',
  '6 7 14 17 24 27',
  '8',
  '9'])

In [10]:
path = set_path + '/'
df_data = pd.DataFrame()
for folder in folder_iters:
    for name in folder:
        dbf = DBF(path + name + '/data.dbf')
        df_temp = pd.DataFrame(iter(dbf))
        df_data = df_data.append(df_temp, ignore_index = True)

        print(name)
        print('\ndata')
        print(df_data.shape)

df_data['ID'] = pd.DataFrame(range(1, 1405))
df_data

1 2

data
(103, 43)
10

data
(130, 43)
11

data
(156, 43)
12 13

data
(198, 43)
15

data
(518, 43)
16 MAR 2020

data
(569, 43)
18 MAR 2020

data
(619, 43)
19-20 MAR 2020

data
(700, 43)
21 22 23

data
(771, 43)
25 26

data
(830, 43)
28 29 30

data
(933, 43)
3

data
(948, 43)
31

data
(972, 43)
4

data
(993, 43)
5

data
(1031, 43)
6 7 14 17 24 27

data
(1309, 43)
8

data
(1347, 43)
9

data
(1375, 43)


,ID,DATE,FEEDER,LINE,CB,CB_STATUS,FROM,TO,TO_DATE,AMP,NAMP,TOCB1,TOCB_DATE,TOCB2,TOCB3,TOCB4,TO1,TO2,TO3,TO4,AMP1,AMP2,AMP3,AMP4,CODE,GROUP,DISTRICT,POLE,LATERAL,ROAD,RELAY,COMPONENT,PHASE,TIMES,LENGTH,TIMEOCB,TIMEFUSE,KVA_HR,LOSS,BATH,TIME_EQ,WEATHER,DATA_PK
0,1,2020-03-01,BB-424,,,,01:43,01:43,2020-03-01,20,25,,2020-03-01,,,,,,,,0,0,0,0,011,F,13,,,,"OCI,EFI,RR",,,,0,0,0,0,0,0.0,0.0,,2020030101:43BB-424
1,2,2020-03-01,WT-435,,,,06:33,06:33,2020-03-01,70,140,,2020-03-01,,,,,,,,0,0,0,0,052,F,17,28,ST.4EE-0818,,"EFI,RR",C22,,,0,0,0,0,0,0.0,0.0,,2020030106:33WT-435
2,3,2020-03-01,KSK-411,,,,06:42,06:42,2020-03-01,40,180,,2020-03-01,,,,,,,,0,0,0,0,051,F,15,,HT.80-178247,,"OCI-B,EFI,RR",C07,B,,0,0,0,0,0,0.0,0.0,,2020030106:42KSK-411
3,4,2020-03-01,BMU-422,,,,06:55,06:55,2020-03-01,40,80,,2020-03-01,,,,,,,,0,0,0,0,052,F,9,,SOI SRISAMIT,,"OCT-YB,EFT,RR",C07,YB,,0,0,0,0,0,0.0,0.0,,2020030106:55BMU-422
4,5,2020-03-01,KM-411,,,,07:19,07:19,2020-03-01,10,20,,2020-03-01,,,,,,,,0,0,0,0,011,F,6,,,,"OCT-B,EFT,RR",,B,,0,0,0,0,0,0.0,0.0,,2020030107:19KM-411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1370,1371,2020-03-09,SA-412,,,,16:39,16:39,2020-03-09,50,225,,2020-03-09,,,,,,,,0,0,0,0,011,F,4,,,,"OCI-T-R,EFI-T,RR",,R,,0,0,0,0,0,0.0,0.0,,2020030916:39SA-412
1371,1372,2020-03-09,BCN-435,,,,17:43,17:43,2020-03-09,170,170,,2020-03-09,,,,,,,,0,0,0,0,191,F,5,,HT.62-026641,,"NOT SHOWN,RR",C07,,,0,0,0,0,0,0.0,0.0,,2020030917:43BCN-435
1372,1373,2020-03-09,TWN-411,,,,18:14,18:14,2020-03-09,50,50,,2020-03-09,,,,,,,,0,0,0,0,011,F,13,,,,"OCI-T-Y,EFI-T,RR",,Y,,0,0,0,0,0,0.0,0.0,,2020030918:14TWN-411
1373,1374,2020-03-09,PP-416,,,,18:33,18:33,2020-03-09,30,50,,2020-03-09,,,,,,,,0,0,0,0,011,F,8,,,,"OCI-T-R,EFI-T,RR",,R,,0,0,0,0,0,0.0,0.0,,2020030918:33PP-416


In [15]:
df_data[['AMP', 'NAMP', 'AMP1', 'AMP2', 'AMP3', 'AMP4', 'DISTRICT', 'LENGTH', 'KVA_HR', 'LOSS', 'BATH']] = df_data[['AMP', 'NAMP', 'AMP1', 'AMP2', 'AMP3', 'AMP4', 'DISTRICT', 'LENGTH', 'KVA_HR', 'LOSS', 'BATH']].fillna(0)

In [8]:
df_data.isna().any()

ID           False
DATE         False
FEEDER       False
LINE         False
CB           False
CB_STATUS    False
FROM         False
TO           False
TO_DATE       True
AMP          False
NAMP         False
TOCB1        False
TOCB_DATE     True
TOCB2        False
TOCB3        False
TOCB4        False
TO1          False
TO2          False
TO3          False
TO4          False
AMP1         False
AMP2         False
AMP3         False
AMP4         False
CODE         False
GROUP        False
DISTRICT     False
POLE         False
LATERAL      False
ROAD         False
RELAY        False
COMPONENT    False
PHASE        False
TIMES        False
LENGTH       False
TIMEOCB      False
TIMEFUSE     False
KVA_HR       False
LOSS         False
BATH         False
TIME_EQ      False
WEATHER      False
DATA_PK      False
dtype: bool

In [ ]:
dbf = DBF('D:/Users/2193033/Desktop/cal_new_indices/022020/1 2 3 4/data.dbf')
df_data = pd.DataFrame(iter(dbf))
# df_data['DATE'].value_counts().sort_index()
df_data['DATE'] = pd.to_datetime(df_data['DATE'], errors='coerce')
df_data = df_data[df_data['DATE'].dt.month==2]
df_data = df_data.sort_values(by='ID', ascending=True)
df_data

In [ ]:
df_data['DATE'] = pd.to_datetime(df_data['DATE'], errors='coerce')
df_data = df_data[df_data['DATE'].dt.month==5]
df_data = df_data.sort_values(by='ID', ascending=True)
df_data

In [11]:
# For data.dbf
import dbf

df_data['DATE'] = pd.to_datetime(df_data['DATE'], errors='coerce')
df_data = df_data[df_data['DATE'].dt.month==3]
df_data = df_data.sort_values(by='ID', ascending=True)

# create an in-memory table
fields = '''
    id N(4,0); date D; feeder C(7); 
    line C(12); cb C(12); cb_status C(1); 
    from C(5); to C(5); to_date D; 
    amp N(3,0); namp N(3,0); tocb1 C(5); 
    tocb_date D; tocb2 C(5); tocb3 C(6); 
    tocb4 C(7); to1 C(8); to2 C(9); 
    to3 C(10); to4 C(11); amp1 N(3,0); 
    amp2 N(3,0); amp3 N(3,0); amp4 N(3,0); 
    code C(3); group C(1); district N(2,0); 
    pole C(3); lateral C(30); road C(30); relay C(27);
    component C(8); phase C(3); times C(3); 
    length N(4,0); timeocb N(6,0); timefuse N(8,0); 
    kva_hr N(10,0); loss N(10,0); bath N(12,2); 
    time_eq N(10,2); weather C(1); data_pk C(40);
'''

# os.mkdir('D:/Users/2193033/Desktop/cal_new_indices/032020/offcial')
table = dbf.Table(
        filename='D:/Users/2193033/Desktop/cal_new_indices/032020/offcial/data.dbf',
        field_specs=fields,
        on_disk=True,
        dbf_type='vfp',
        codepage=124
        )
table.open(mode=dbf.READ_WRITE)

for index, row in df_data.iterrows(): 
#     print (index, tuple(row))
    table.append(tuple(row))

print(table)
table.close()


        Table:         D:/Users/2193033/Desktop/cal_new_indices/032020/offcial/data.dbf
        Type:          Visual Foxpro
        Codepage:      cp874 (Thai (ANSI/OEM))
        Status:        DbfStatus.READ_WRITE
        Last updated:  2020-12-22
        Record count:  1375
        Field count:   43
        Record length: 361 
        --Fields--
          0) b'ID N(4,0)'
          1) b'DATE D'
          2) b'FEEDER C(7)'
          3) b'LINE C(12)'
          4) b'CB C(12)'
          5) b'CB_STATUS C(1)'
          6) b'FROM C(5)'
          7) b'TO C(5)'
          8) b'TO_DATE D'
          9) b'AMP N(3,0)'
         10) b'NAMP N(3,0)'
         11) b'TOCB1 C(5)'
         12) b'TOCB_DATE D'
         13) b'TOCB2 C(5)'
         14) b'TOCB3 C(6)'
         15) b'TOCB4 C(7)'
         16) b'TO1 C(8)'
         17) b'TO2 C(9)'
         18) b'TO3 C(10)'
         19) b'TO4 C(11)'
         20) b'AMP1 N(3,0)'
         21) b'AMP2 N(3,0)'
         22) b'AMP3 N(3,0)'
         23) b'AMP4 N(3,0)'
       

### switching.dbf

In [12]:
path = set_path + '/'
df_sw = pd.DataFrame()
for folder in folder_iters:
    for name in folder:
        dbf = DBF(path + name + '/switching.dbf')
        df_temp = pd.DataFrame(iter(dbf))
        df_sw = df_sw.append(df_temp, ignore_index = True)

        print(name)
        print('switching')
        print(df_sw.shape, '\n')

df_sw.drop(columns=['_NullFlags'], inplace=True)
df_sw['tmp_date'] = df_sw['DATA_PK'].map(lambda x: str(x)[0:8])
df_sw['month'] = df_sw['DATA_PK'].map(lambda x: int(str(x)[4:6]))
df_sw['day'] = df_sw['DATA_PK'].map(lambda x: int(str(x)[6:8]))
print(df_sw['tmp_date'].value_counts().sort_index())
print('\n', df_sw.shape)

1 2
switching
(3, 14) 

10
switching
(15, 14) 

11
switching
(15, 14) 

12 13
switching
(22, 14) 

15
switching
(121, 14) 

16 MAR 2020
switching
(127, 14) 

18 MAR 2020
switching
(129, 14) 

19-20 MAR 2020
switching
(132, 14) 

21 22 23
switching
(141, 14) 

25 26
switching
(144, 14) 

28 29 30
switching
(147, 14) 

3
switching
(147, 14) 

31
switching
(151, 14) 

4
switching
(157, 14) 

5
switching
(174, 14) 

6 7 14 17 24 27
switching
(269, 14) 

8
switching
(284, 14) 

9
switching
(290, 14) 

20200302     3
20200304     6
20200305    17
20200306    10
20200307    69
20200308    15
20200309     6
20200310    12
20200312     5
20200313     2
20200314     5
20200315    99
20200316     6
20200317     2
20200318     2
20200320     3
20200323     9
20200324     6
20200326     3
20200327     3
20200330     3
20200331     4
Name: tmp_date, dtype: int64

 (290, 16)


In [ ]:
dbf = DBF('D:/Users/2193033/Desktop/cal_new_indices/022020/4/switching.dbf')
df_sw = pd.DataFrame(iter(dbf))
df_sw.drop(columns=['_NullFlags'], inplace=True)
df_sw['tmp_date'] = df_sw['DATA_PK'].map(lambda x: str(x)[0:8])
df_sw['month'] = df_sw['DATA_PK'].map(lambda x: int(str(x)[4:6]))
df_sw['day'] = df_sw['DATA_PK'].map(lambda x: int(str(x)[6:8]))
print(df_sw['tmp_date'].value_counts().sort_index())
print('\n', df_sw.shape)

In [ ]:
dbf = DBF('D:/Users/2193033/Desktop/cal_new_indices/012020/23/switching.dbf')
df_sw_2 = pd.DataFrame(iter(dbf))
df_sw_2.drop(columns=['_NullFlags'], inplace=True)
df_sw_2['tmp_date'] = df_sw_2['DATA_PK'].map(lambda x: str(x)[0:8])
df_sw_2['month'] = df_sw_2['DATA_PK'].map(lambda x: int(str(x)[4:6]))
df_sw_2['day'] = df_sw_2['DATA_PK'].map(lambda x: int(str(x)[6:8]))
print(df_sw_2['DATE_FROM'].value_counts().sort_index())
print('\n', df_sw_2.shape)

In [ ]:
df_sw = df_sw.append(df_sw_2, ignore_index = True)
df_sw

In [13]:
# For switching.dbf

import dbf

df_sw = df_sw[df_sw['month']==3]
df_sw = df_sw.drop(['tmp_date', 'month', 'day'], axis=1)
df_sw['DURATION'].fillna(0, inplace=True)

# create an in-memory table
fields = '''
    id_sw N(16,0); int_step C(2); res_step C(3); 
    date_from D; time_from C(5); time_to C(5); 
    date_to D; duration N(10,0) null; poi1 C(254); 
    poi2 C(254); xfeeder C(7); exclude L; 
    data_pk C(40);
'''

table = dbf.Table(
        filename='D:/Users/2193033/Desktop/cal_new_indices/032020/offcial/switching.dbf',
        field_specs=fields,
        on_disk=True,
        dbf_type='vfp',
        codepage=124
        )
table.open(mode=dbf.READ_WRITE)

for index, row in df_sw.iterrows(): 
#     print (index, tuple(row))
    table.append(tuple(row))

print(table)
table.close()


        Table:         D:/Users/2193033/Desktop/cal_new_indices/032020/offcial/switching.dbf
        Type:          Visual Foxpro
        Codepage:      cp874 (Thai (ANSI/OEM))
        Status:        DbfStatus.READ_WRITE
        Last updated:  2020-12-22
        Record count:  290
        Field count:   13
        Record length: 615 
        --Fields--
          0) b'ID_SW N(16,0)'
          1) b'INT_STEP C(2)'
          2) b'RES_STEP C(3)'
          3) b'DATE_FROM D'
          4) b'TIME_FROM C(5)'
          5) b'TIME_TO C(5)'
          6) b'DATE_TO D'
          7) b'DURATION N(10,0) null'
          8) b'POI1 C(254)'
          9) b'POI2 C(254)'
         10) b'XFEEDER C(7)'
         11) b'EXCLUDE L'
         12) b'DATA_PK C(40)'



### customer.dbf

In [14]:
path = set_path + '/'
df_cus = pd.DataFrame()
for folder in folder_iters:
    for name in folder:
        dbf = DBF(path + name + '/customer.dbf')
        df_temp = pd.DataFrame(iter(dbf))
        df_cus = df_cus.append(df_temp, ignore_index = True)

        print(name)
        print('customer')
        print(df_cus.shape, '\n')

df_cus['tmp_date'] = df_cus['ID_CUST'].map(lambda x: str(x)[1:9])
df_cus['month'] = df_cus['ID_CUST'].map(lambda x: int(str(x)[5:7]))
df_cus['day'] = df_cus['ID_CUST'].map(lambda x: int(str(x)[7:9]))
print(df_cus['tmp_date'].value_counts().sort_index())
print('\n', df_cus.shape)

1 2
customer
(3, 9) 

10
customer
(15, 9) 

11
customer
(15, 9) 

12 13
customer
(22, 9) 

15
customer
(144, 9) 

16 MAR 2020
customer
(152, 9) 

18 MAR 2020
customer
(154, 9) 

19-20 MAR 2020
customer
(157, 9) 

21 22 23
customer
(169, 9) 

25 26
customer
(172, 9) 

28 29 30
customer
(175, 9) 

3
customer
(175, 9) 

31
customer
(179, 9) 

4
customer
(185, 9) 

5
customer
(206, 9) 

6 7 14 17 24 27
customer
(317, 9) 

8
customer
(336, 9) 

9
customer
(342, 9) 

20200302      3
20200304      6
20200305     21
20200306     16
20200307     77
20200308     19
20200309      6
20200310     12
20200312      5
20200313      2
20200314      7
20200315    122
20200316      8
20200317      2
20200318      2
20200320      3
20200323     12
20200324      6
20200326      3
20200327      3
20200330      3
20200331      4
Name: tmp_date, dtype: int64

 (342, 12)


In [ ]:
dbf = DBF('D:/Users/2193033/Desktop/cal_new_indices/022020/4/customer.dbf')
df_cus = pd.DataFrame(iter(dbf))
df_cus['tmp_date'] = df_cus['ID_CUST'].map(lambda x: str(x)[1:9])
df_cus['month'] = df_cus['ID_CUST'].map(lambda x: int(str(x)[5:7]))
df_cus['day'] = df_cus['ID_CUST'].map(lambda x: int(str(x)[7:9]))
print(df_cus['tmp_date'].value_counts().sort_index())
print('\n', df_cus.shape)

In [ ]:
dbf = DBF('D:/Users/2193033/Desktop/cal_new_indices/012020/23/customer.dbf')
df_cus_2 = pd.DataFrame(iter(dbf))
df_cus_2['tmp_date'] = df_cus_2['ID_CUST'].map(lambda x: str(x)[1:9])
df_cus_2['month'] = df_cus_2['ID_CUST'].map(lambda x: int(str(x)[5:7]))
df_cus_2['day'] = df_cus_2['ID_CUST'].map(lambda x: int(str(x)[7:9]))
print(df_cus_2['tmp_date'].value_counts().sort_index())
print('\n', df_cus_2.shape)

In [ ]:
df_cus = df_cus.append(df_cus_2, ignore_index = True)
df_cus

In [15]:
# For customer.dbf

import dbf


df_cus = df_cus[df_cus['month']==3]
df_cus = df_cus.drop(columns=['tmp_date', 'month', 'day'])
df_cus.loc[df_cus['CUST_RES'].isnull(), 'CUST_NUM'] = df_cus.loc[df_cus['CUST_RES'].isnull(), 'CUST_INT']
df_cus.loc[df_cus['CUST_RES'].notnull(), 'CUST_NUM'] = df_cus.loc[df_cus['CUST_RES'].notnull(), 'CUST_RES']
df_cus.fillna(0, inplace=True)

# create an in-memory table
fields = '''
    id_cust N(16,0); id_sw N(16,0); id_dist N(2,0); 
    id_subarea C(1); id_provinc C(1); cust_res N(10,0); 
    cust_int N(10,0); cust_num N(12,0); cust_min N(12,2);
'''

table = dbf.Table(
        filename='D:/Users/2193033/Desktop/cal_new_indices/032020/offcial/customer.dbf',
        field_specs=fields,
        on_disk=True,
        dbf_type='vfp',
        codepage=124
        )
table.open(mode=dbf.READ_WRITE)

for index, row in df_cus.iterrows(): 
#     print (index, tuple(row))
    table.append(tuple(row))

print(table)
table.close()


        Table:         D:/Users/2193033/Desktop/cal_new_indices/032020/offcial/customer.dbf
        Type:          Visual Foxpro
        Codepage:      cp874 (Thai (ANSI/OEM))
        Status:        DbfStatus.READ_WRITE
        Last updated:  2020-12-22
        Record count:  342
        Field count:   9
        Record length: 81 
        --Fields--
          0) b'ID_CUST N(16,0)'
          1) b'ID_SW N(16,0)'
          2) b'ID_DIST N(2,0)'
          3) b'ID_SUBAREA C(1)'
          4) b'ID_PROVINC C(1)'
          5) b'CUST_RES N(10,0)'
          6) b'CUST_INT N(10,0)'
          7) b'CUST_NUM N(12,0)'
          8) b'CUST_MIN N(12,2)'



In [ ]:
df_gather = df_data.merge(df_sw, how='inner', on='DATA_PK')
df_gather = df_gather.merge(df_cus, how='left', on='ID_SW')
df_gather

In [ ]:
df_gather[(df_gather['CUST_INT'].isnull()) & (df_gather['CUST_RES'].isnull())]

In [ ]:
import datetime
import dbf

# create an in-memory table
table = dbf.Table(
        filename='test',
        field_specs='name C(25); age N(3,0); birth D; qualified L',
        on_disk=False,
        )
table.open(mode=dbf.READ_WRITE)

# # add some records to it
# for datum in (
#         ('Spanky', 7, dbf.Date.fromymd('20010315'), False),
#         ('Spunky', 23, dbf.Date(1989, 7, 23), True),
#         ('Sparky', 99, dbf.Date(), dbf.Unknown),
#         ):
#     table.append(datum)

# add some records to it
df = pd.DataFrame(( 
        ('Spanky', 7, datetime.date(2001, 3, 15), False),
        ('Spunky', 23, datetime.date(1989, 7, 23), True),
        ('Sparky', 99, None, None) 
     ))
for index, row in df.iterrows(): 
#     print (type(index), type(row))
    table.append(tuple(row))

# iterate over the table, and print the records
for record in table:
    print(record)
    print('--------')
    print(record[0:4])
    print([record.name, record.age, record.birth, record.qualified])
    print('--------')

# make a copy of the test table (structure, not data)
custom = table.new(
        filename='test_on_disk',
        default_data_types=dict(C=dbf.Char, D=dbf.Date, L=dbf.Logical),
        )

# automatically opened and closed
with custom:
    # copy records from test to custom
    for record in table:
        custom.append(record)
    # modify each record in custom (could have done this in prior step)
    for record in custom:
        dbf.write(record, name=record.name.upper())
        # and print the modified record
        print(record)
        print('--------')
        print(record[0:3])
        print([record.name, record.age, record.birth])
        print('--------')

table.close()

In [ ]:
# df_data['DATE'] = pd.to_datetime(df_data['DATE'], errors='coerce')
df_data[df_data['DATE'].dt.day==22][['FEEDER', 'LINE', 'CB', 'TIMEOCB']]